# Feature extraction
At this stage, the raw data should be transform into a standard time series format. This codes is to first transform the data into a "cesium" format, and then extract the feature from it.

In [9]:
#load libraries
from datetime import datetime, timedelta
from dateutil import parser
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
from cesium import featurize

#specify the data dir
data_file = '/Users/leeo/Desktop/KI2/7.master_thesis/1.data/4.ts_format/17273_ts/17273_14-11-17.csv'

In [10]:
data = pd.read_csv(data_file, header=0, index_col=0)
data.index = pd.to_datetime(data.index)
data.head()

,HR,Resp,Comments
Date_Time,,,
2017-11-14 08:30:53.761200,161.2035,2.046443,NaN
2017-11-14 08:30:54.261200,149.3424,2.931300,NaN
2017-11-14 08:30:54.761200,139.5089,2.540741,NaN
2017-11-14 08:30:55.261200,170.1940,2.610959,NaN
2017-11-14 08:30:55.761200,162.9608,2.745577,NaN


## Data slice : slice with windows

In [11]:
#check the comments for label
data["Comments"][data["Comments"].isnull()==False]

Date_Time
2017-11-14 08:43:00.261200                  #* 600ng LPS 
2017-11-14 09:07:52.261200    #* finish giving 600ng lps 
Name: Comments, dtype: object

<img src="pictures/data_slicing_concepts.jpg" width="800" height="400">

In [12]:
t0 = pd.to_datetime("2017-11-14 08:43:00.261200") #starting time for sepsis
delta_t = pd.Timedelta("1T")
time_frame = pd.Timedelta("10T")
time_window = pd.Timedelta("1h")
n = int((time_window - time_frame)/delta_t +1)
t1 = t0 + pd.Timedelta("1.5h") #starting time for sepsis recover

print("The starting time of sepsis status is {}, we choose {} \nas time window, where each time frame rolls in {}, and therefore there is \n{} time frames in total ".format(
t0, time_window, delta_t, n))

The starting time of sepsis status is 2017-11-14 08:43:00.261200, we choose 0 days 01:00:00 
as time window, where each time frame rolls in 0 days 00:01:00, and therefore there is 
51 time frames in total 


In [13]:
cesium_times = []
cesium_values = []
for i in range(n):
    t_0_start = t0 + delta_t*i
    t_0_stop = t_0_start + time_frame - pd.Timedelta("0.5s")
    cesium_df = data[t_0_start:t_0_stop]
    cesium_t = cesium_df.loc[t_0_start:t_0_stop].index.to_numpy()
    cesium_HR = cesium_df.loc[t_0_start:t_0_stop].HR.to_numpy()
    cesium_Resp = cesium_df.loc[t_0_start:t_0_stop].Resp.to_numpy()
    
    #list for labels?
    cesium_values.append(np.array([cesium_HR,cesium_Resp]))
    cesium_times.append(np.array([cesium_t.astype("float"),cesium_t.astype("float")]))
if len(cesium_values) != n:
    raise Exception('The time window is incontinous, please check the experiment data.')
if cesium_values[0].shape != (2, time_frame.total_seconds()/pd.Timedelta("0.5s").total_seconds()):
    raise Exception('Check the HR or Resp values, if there any missing values, or you forget to include one of them.')
    
print("Within the {} of cesium_values, there are {} objects, and each object is a {} with the shape of {}. Example: \n{}".format(
type(cesium_values), len(cesium_values), type(cesium_values[0]), cesium_values[0].shape,cesium_values[0]))

Within the <class 'list'> of cesium_values, there are 51 objects, and each object is a <class 'numpy.ndarray'> with the shape of (2, 1200). Example: 
[[162.9751    163.9367    145.1801    ... 177.4922    174.2096
  173.2778   ]
 [  0.7797833   2.247328    1.728741  ...   4.206926    4.13492
    4.254256 ]]


In [20]:
#set the feature wants to extract
features_to_use = ["amplitude",
                   "percent_beyond_1_std",
                   "maximum",
                   "max_slope",
                   "median",
                   "median_absolute_deviation",
                   "percent_close_to_median",
                   "minimum" ,
                   "skew",
                   "std",
                   "weighted_average"]

fset_cesium = featurize.featurize_time_series(times = cesium_times,
                                              values = cesium_values,
                                              features_to_use=features_to_use)
fset_cesium


feature    amplitude           percent_beyond_1_std               maximum  \
channel            0         1                    0         1           0   
0        7351.956105  6.768671             0.002500  0.325833  14721.6600   
1        2789.075605  6.768671             0.019167  0.348333   5595.8990   
2        2789.075605  6.768671             0.015833  0.345833   5595.8990   
3        1909.627605  5.264268             0.041667  0.338333   3837.0030   
4        4949.475105  5.264268             0.010833  0.287500   9916.6980   
5        4918.490440  4.936377             0.011667  0.210000   9916.6980   
6        4918.490440  3.628561             0.009167  0.272500   9916.6980   
7        4946.727430  2.794732             0.008333  0.311667   9916.6980   
8        4946.727430  2.578297             0.005000  0.340833   9916.6980   
9        4946.727430  2.578297             0.005000  0.327500   9916.6980   
10       4946.727430  2.578297             0.005000  0.340000   9916.6980   
11       4946.727430  2.698575             0.006667  0.312500   9916.6980   
12       4946.727430  2.698575             0.006667  0.304167   9916.6980   
13       6388.298430  2.856259             0.004167  0.283333  12799.8400   
14       6388.298430  2.856259             0.002500  0.289167  12799.8400   
15       6388.298430  2.447183             0.002500  0.298333  12799.8400   
16       6388.298430  2.533476             0.002500  0.308333  12799.8400   
17       6372.086120  2.533476             0.002500  0.318333  12799.8400   
18       6372.086120  3.396463             0.001667  0.297500  12799.8400   
19       6372.086120  4.133658             0.001667  0.277500  12799.8400   
20       6372.086120  5.950147             0.001667  0.227500  12799.8400   
21       6372.086120  5.950147             0.001667  0.269167  12799.8400   
22       6372.086120  5.950147             0.001667  0.305833  12799.8400   
23        153.265970  5.950147             0.152500  0.307500    362.1997   
24        153.265970  5.950147             0.156667  0.292500    362.1997   
25        153.265970  5.950147             0.170833  0.268333    362.1997   
26       2044.818980  5.950147             0.003333  0.232500   4142.8180   
27       2044.818980  5.950147             0.002500  0.209167   4142.8180   
28       2044.818980  4.377326             0.002500  0.226667   4142.8180   
29       2044.818980  4.377326             0.002500  0.235833   4142.8180   
30       2044.818980  4.284467             0.002500  0.258333   4142.8180   
31       2044.818980  3.964657             0.002500  0.280833   4142.8180   
32       2044.818980  2.996547             0.002500  0.247500   4142.8180   
33       2044.818980  2.996547             0.003333  0.252500   4142.8180   
34       2044.818980  2.996547             0.003333  0.320000   4142.8180   
35       2044.818980  2.968795             0.004167  0.364167   4142.8180   
36       1231.185000  3.732891             0.004167  0.306667   2567.3130   
37       1231.185000  3.732891             0.004167  0.338333   2567.3130   
38       1245.635525  3.732891             0.005000  0.401667   2567.3130   
39       1245.635525  3.732891             0.005000  0.383333   2567.3130   
40       2968.950525  3.732891             0.004167  0.328333   6013.9430   
41       2968.950525  3.291369             0.010833  0.306667   6013.9430   
42       2968.950525  2.414025             0.013333  0.365000   6013.9430   
43       4367.366525  3.909007             0.020833  0.310833   8810.7750   
44       4956.743285  3.909007             0.030000  0.359167   9936.1320   
45       5030.707285  3.909007             0.031667  0.416667  10084.0600   
46       5030.707285  3.852316             0.033333  0.403333  10084.0600   
47       5030.707285  3.852316             0.034167  0.398333  10084.0600   
48       5030.707285  3.852316             0.034167  0.360833  10084.0600   
49       5030.707285  3.852316             0.034167  0.273333  10084.0600